In [34]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
import time
import datetime

In [35]:
data = pd.read_csv('./data/round1_ijcai_18_train_20180301.txt', sep=' ')
data.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [36]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade'],
      dtype='object')

In [37]:
def timestamp_datetime(date):
    format = "%Y-%m-%d %H:%M:%S"
    date = time.localtime(date)
    dt = time.strftime(format, date)
    dt = datetime.datetime.strptime(dt, format)
    return dt

In [38]:
data['time'] = data.context_timestamp.apply(timestamp_datetime)

In [42]:
data['day'] = data.time.apply(lambda x: x.day)
data['hour'] = data.time.apply(lambda x: x.hour)

In [44]:
user_query_day = data.groupby(['user_id', 'day']).size().reset_index().rename(columns={0: 'user_query_day'})
user_query_day.head()

,user_id,day,user_query_day
0,24779788309075,19,3
1,36134987234568,22,1
2,59341486148291,22,1
3,179317972644611,22,7
4,179317972644611,23,1


In [45]:
data = pd.merge(data, user_query_day, 'left', on=['user_id', 'day'])

In [49]:
user_query_day_hour = data.groupby(['user_id', 'day', 'hour'])\
                            .size().reset_index()\
                            .rename(columns={0: 'user_query_day_hour'})
user_query_day_hour.head()

,user_id,day,hour,user_query_day_hour
0,24779788309075,19,21,1
1,24779788309075,19,22,2
2,36134987234568,22,12,1
3,59341486148291,22,19,1
4,179317972644611,22,21,7


In [50]:
data = pd.merge(data, user_query_day_hour, 'left', on=['user_id', 'day', 'hour'])

In [51]:
data.shape

(478138, 32)

In [52]:
data.day.value_counts()

18    78268
21    71199
19    70931
20    68387
22    68318
23    63614
24    57421
Name: day, dtype: int64

In [53]:
train = data.loc[data.day < 24]
test = data.loc[data.day == 24]

In [55]:
features = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
            'item_collected_level', 'item_pv_level', 'user_gender_id', 'user_occupation_id',
            'user_age_level', 'user_star_level', 'user_query_day', 'user_query_day_hour',
            'context_page_id', 'hour', 'shop_id', 'shop_review_num_level', 'shop_star_level',
            'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 
            'shop_score_description'
            ]

In [56]:
target = ['is_trade']

In [84]:
clf = lgb.LGBMClassifier(num_leaves=80, max_depth=8, 
                         n_estimators=200, n_jobs=20)

In [85]:
clf.fit(train[features], train[target], feature_name=features,
       categorical_feature=['user_gender_id'])

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['user_gender_id']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=8, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
        n_jobs=20, num_leaves=80, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [86]:
test['lgb_predict'] = clf.predict_proba(test[features],)[:, 1]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
print(log_loss(test[target], test['lgb_predict']))

0.08306911639192413


In [88]:
test['lgb_predict'].sort_values(ascending=False)

437749    0.616655
454094    0.595862
462973    0.413258
457555    0.383991
445268    0.345290
437671    0.303434
448624    0.294495
476470    0.281060
454084    0.274056
448592    0.267885
455243    0.267695
454088    0.266316
475260    0.264528
459065    0.250082
457554    0.243666
424550    0.234068
422666    0.233432
424537    0.233190
450675    0.232739
475267    0.231767
475262    0.230603
429218    0.229305
422641    0.229256
424538    0.223708
437380    0.221822
447829    0.221421
446595    0.220910
451566    0.218942
472734    0.218003
475645    0.217143
            ...   
466587    0.000182
476738    0.000178
420888    0.000177
426413    0.000176
475899    0.000168
469358    0.000167
477928    0.000165
440780    0.000163
432838    0.000160
452090    0.000160
459785    0.000159
473287    0.000155
456146    0.000151
456630    0.000145
447612    0.000140
471303    0.000139
434900    0.000131
453573    0.000131
461657    0.000130
430277    0.000125
478007    0.000115
471305    0.